# Machine Learning Classification Example Using Exasol

### Introduction

This example demonstrates how machine learning techniques can be used inside Exasol.

It was created using the publicly available dataset from the Industrial Challenge at the [15th International Symposium on Intelligent Data Analysis](http://ida2016.blogs.dsv.su.se/) (IDA) in 2016. The dataset, provided by Scania CV AB, consists of real data from heavy Scania trucks during normal operation. For details, please see the data description file provided with the data.

### Purpose

This example is to show how you can use machine learning tools with Exasol. Therefore, we will not discuss the machine learning details such as classifier selection and tuning. Because there are many different machine learning methods, choosing a "good" one is highly dependent on the problem to be solved and the data. Rather, we want to demonstrate how you can more effectively use <b>your models</b> with <b>your data</b> in Exasol.

### Prerequisites

The intended audience of this article is assumed to have a basic understanding of the following.
* Exasol, in particular UDFs
* Machine learning methods
* Python programming, including
  * Scikit-learn
  * Pandas
  * NumPy

### Problem Description

The purpose of the challenge was to best predict which failures are related to a specific component of a truck's air pressure system (APS) as opposed to failures unrelated to the APS. Specifically, the following cost metric was given, which was to be minimized.

$$cost_{total}=cost_{FP}\cdot{FP} + cost_{FN}\cdot{FN}$$
where  
$FP$ is the number of false positives (predicted APS failure, but really isn't),  
$FN$ is the number of false negatives (predicted non-APS failure, but really is),  
$cost_{FP}=10$ is the cost of an unnecessary check by a mechanic, and  
$cost_{FN}=500$ is the cost of not checking a faulty truck and possibly causing a breakdown.

So from the cost metric, we can see that a preventative check is much cheaper (50x) than a repair after a breakdown, which makes sense.

### Setup

Here, we specify some basic information, which is used throughout this example. In particular, the location, user, and password for the Exasol host(s) and EXABucket are specified, as well as the scripting language to be used.

In [ ]:
EXASOL_HOST = "localhost:8563"
EXASOL_USER = "sys"
EXASOL_PASSWORD = "pw"
EXASOL_BUCKETFS_HOST = "localhost:2581"
EXASOL_BUCKETFS_USER = "w"
EXASOL_BUCKETFS_PASSWORD = "pw"
EXASOL_SCRIPT_LANGUAGES = "SKLEARN_PYTHON=localzmq+protobuf:///bucketfs1/udfs/scikit_learn?lang=python#buckets/bucketfs1/udfs/scikit_learn/exaudf/exaudfclient";

### Get the Data

To begin, we download the [IDA 2016 Challenge dataset](https://archive.ics.uci.edu/ml/datasets/IDA2016Challenge) (20MB) from the [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php) and import it into training and test DataFrames.

Because the ZIP file contains a data description file and both the training and test data, which must be kept separate, we cannot import the file directly into Exasol using Exasol's IMPORT statement. Instead, we first read the two data files into separate local DataFrames.

In [ ]:
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

import pandas as pd

DATA_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00414/to_uci.zip"
TRAINING_FILE = "to_uci/aps_failure_training_set.csv"
TEST_FILE = "to_uci/aps_failure_test_set.csv"

# Data is preceeded with a 20-line header (copyright & license)
NUM_SKIP_ROWS = 20
NA_VALUE = "na"

resp = urlopen(DATA_URL)
with ZipFile(BytesIO(resp.read())) as z:
    train_set = pd.read_csv(TRAINING_FILE, skiprows=NUM_SKIP_ROWS, na_values=NA_VALUE)
    test_set = pd.read_csv(TEST_FILE, skiprows=NUM_SKIP_ROWS, na_values=NA_VALUE)

### Load the Data into Exasol

We can now load the data from the DataFrames into Exasol tables. By having a quick look at the data and/or reading the provided data description file, we can see that the first data column is the class label ('neg'/'pos') and can be stored in a `VARCHAR(3)` column. The other data columns are all numerical features which can be stored in `DECIMAL(18, 2)` columns.

For this example, we create an Exasol schema named `IDA`, in which everything will be stored. Then, we load the training and test data from the DataFrames into two tables named `TRAIN` and `TEST`, respectively. For this step, we use the very convenient [pyexasol](https://github.com/badoo/pyexasol) module.

In [ ]:
import pyexasol

# Create Exasol connection
conn = pyexasol.connect(dsn=EXASOL_HOST, user=EXASOL_USER, password=EXASOL_PASSWORD, compression=True)

# Create schema
conn.execute("CREATE SCHEMA IF NOT EXISTS IDA")
conn.execute("OPEN SCHEMA IDA")

# Create tables for data
column_names = list(train_set.columns)
column_types = ["VARCHAR(3)"] + ["DECIMAL(18,2)"] * (len(column_names) - 1)
column_desc = [" ".join(t) for t in zip(column_names, column_types)]

conn.execute("CREATE OR REPLACE TABLE IDA.TRAIN(" + ", ".join(column_desc) + ")")
conn.execute("CREATE OR REPLACE TABLE IDA.TEST LIKE IDA.TRAIN")

# Import data into Exasol
conn.import_from_pandas(train_set, "TRAIN")
print(f"Imported {conn.last_statement().rowcount()} rows into IDA.TRAIN.")
conn.import_from_pandas(test_set, "TEST")
print(f"Imported {conn.last_statement().rowcount()} rows into IDA.TEST.")

# Close Exasol connection
conn.close()

### Examine the Data

After loading the data into Exasol, we may first want to get a feel for the data before creating a classifier. There are many different ways to do so, such as data visualization, viewing basic statistical information, examining feature correlation, etc.

In the example below, we examine the training data's basic statistical information using `pandas.DataFrame.describe()` and `pandas.DataFrame.var()` and output the combinded results for the first five columns.

In [ ]:
import textwrap

# Create Exasol connection
conn = pyexasol.connect(dsn=EXASOL_HOST, user=EXASOL_USER, password=EXASOL_PASSWORD, compression=True)
conn.execute(f"ALTER SESSION SET SCRIPT_LANGUAGES='{EXASOL_SCRIPT_LANGUAGES}'")

# Create script output column descriptions
# Numeric data
out_column_types = ["DOUBLE"] * len(column_names)
out_column_desc = [" ".join(t) for t in zip(column_names, out_column_types)]

# Create script to run pandas.DataFrame.describe() and pandas.DataFrame.var() in Exasol
sql = textwrap.dedent(f"""\
CREATE OR REPLACE SKLEARN_PYTHON SET SCRIPT IDA.DF_DESCRIBE({", ".join(column_desc)})
EMITS ({", ".join(out_column_desc)}) AS

import numpy as np
import pandas as pd

# Create DataFrame
def create_df(ctx):
    df = []
    col_names = [col.name for col in exa.meta.input_columns]
    while True:
        row = []
        for i in range(0, exa.meta.input_column_count):
            row.append(ctx[i])
        df.append(row)
        if not ctx.next():
            break;
    df = pd.DataFrame(df, columns=col_names)
    return df

# Convert NumPy value to native Python type
def np_numeric_as_scalar(x):
    return [np.asscalar(x[i]) for i in range(0, len(x))]

def get_stats(X):
    # Replace 'neg'/'pos' with 0/1
    X.loc[:, 'class'] = X.loc[:, 'class'].replace({{'neg': 0, 'pos': 1}})
    # Convert all columns to numeric data types
    X = X.apply(pd.to_numeric)

    # Get DataFrame stats
    X_describe = X.describe()

    # Get DataFrame variance
    X_var = X.var()
    X_var.name = 'var'

    # Append variance to stats
    return X_describe.append(X_var)

def run(ctx):
    df = create_df(ctx)

    # Calculate statistics info
    df = get_stats(df)

    # Output data description
    for i in range(0, df.shape[0]):
        out_list = np_numeric_as_scalar(df.iloc[i, :])
        ctx.emit(*out_list)
/
""")

conn.execute(sql)

# Create table "TRAIN_DESCRIPTION" to hold the description output
sql = textwrap.dedent(f"""\
CREATE OR REPLACE TABLE IDA.TRAIN_DESCRIPTION AS
    SELECT IDA.DF_DESCRIBE({", ".join(column_names)}) from IDA.TRAIN
""")
conn.execute(sql)

# Create local data frame from the "TRAIN_DESCRIPTION" table
train_desc = conn.export_to_pandas("SELECT * FROM IDA.TRAIN_DESCRIPTION")
train_desc.index = ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max', 'var']

# Print first 5 columns, for example
print(train_desc.iloc[:, 0:5])

# Close Exasol connection
conn.close()

### Transform the Data

Looking at the statistics summary from the previous step, we can see, for example, that some features have missing values and that the means and variances of the features differ greatly. Because of this, it is most likely a good idea to transform the data.

Depending on which classifier one plans to use, among other things, there are many different techniques one may use to transform the data, such as feature scaling and extraction.

In this example, we first use imputation to replace missing values with the median value of that feature. Then, we scale the data such that each feature is normally distributed (with zero mean and unit variance). These are very simple transformations that work fairly well with many learning algorithms.

In the script, the transformer (actually a transform pipeline), is fitted to the training data and then used to transform it. Then, the transformer object is saved to the specified EXABucket for future use. Finally, the transformed training data is outputted and stored in the table `TRAIN_TRANSFORMED`.

The same script is then called again with the test data. Since the test data should be transformed exactly as the training data, the transformer which was previously saved is simply loaded from the EXABucket and used to transform the test data. The transformed test data is outputted and stored in the table `TEST_TRANSFORMED`.

In [ ]:
# Create Exasol connection
conn = pyexasol.connect(dsn=EXASOL_HOST, user=EXASOL_USER, password=EXASOL_PASSWORD, compression=True)
conn.execute(f"ALTER SESSION SET SCRIPT_LANGUAGES='{EXASOL_SCRIPT_LANGUAGES}'")

# Create script output column descriptions
# One class label, numeric data
out_column_types = ["INT"] + ["DOUBLE"] * (len(column_names) - 1)
out_column_desc = [" ".join(t) for t in zip(column_names, out_column_types)]

# Specify EXABucket service (for storing the transformer)
bucketfs_service = "/buckets/bucketfs1"
# Specify transformer path in the EXABucket service
transformer_path = "/udfs/transform_pipeline.pkl"

# Create script to run pandas.DataFrame.describe() and pandas.DataFrame.var() in Exasol
sql = textwrap.dedent(f"""\
CREATE OR REPLACE SKLEARN_PYTHON SET SCRIPT
IDA.DF_TRANSFORM(fit_transformer BOOL, transformer_path VARCHAR(200), {", ".join(column_desc)})
EMITS ({", ".join(out_column_desc)}) AS

import numpy as np
import pandas as pd
import pycurl
import uuid

from sklearn.externals import joblib
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler

# Create DataFrame
def create_df(ctx, start_col, num_cols):
    df = []
    col_names = [col.name for col in exa.meta.input_columns[start_col : num_cols + num_cols]]
    while True:
        row = []
        for i in range(start_col, start_col + num_cols):
            row.append(ctx[i])
        df.append(row)
        if not ctx.next():
            break;
    df = pd.DataFrame(df, columns=col_names)
    return df

# Upload file to EXABucket
def upload_file_bucketfs(object, path):
    temp_file = "/tmp/" + str(uuid.uuid4().hex + ".pkl")
    joblib.dump(object, temp_file, compress=True)

    with open(temp_file, "rb") as f:
        url = "https://{EXASOL_BUCKETFS_USER}:{EXASOL_BUCKETFS_PASSWORD}@{EXASOL_BUCKETFS_HOST}" + path
        curl = pycurl.Curl()
        curl.setopt(pycurl.URL, url)
        curl.setopt(pycurl.SSL_VERIFYPEER, 0)   
        curl.setopt(pycurl.SSL_VERIFYHOST, 0)
        curl.setopt(curl.UPLOAD, 1)
        curl.setopt(curl.READDATA, f)
        curl.perform()
        curl.close()

# Convert NumPy values to native Python type
def np_numeric_as_scalar(x):
    return [np.asscalar(x[i]) for i in range(0, len(x))]

# Transform DataFrame
def transform_dataframe(X, class_col_name, fit_transformer=False, save_path=None):
    y = X.loc[:, class_col_name]
    X_data = X.loc[:, X.columns != class_col_name]

    # Replace 'neg'/'pos' with 0/1
    y = y.replace({{'neg': 0, 'pos': 1}})

    # Convert columns to numeric data types
    X_data = X_data.apply(pd.to_numeric)

    if fit_transformer:
        # Fit transformer and transform data
        transformer = Pipeline ([
            ('imputer', Imputer(strategy="median")),
            ('scaler', StandardScaler())
        ])
        X_data_transformed = transformer.fit_transform(X_data)
        if save_path:
            # Save transformer
            upload_file_bucketfs(transformer, save_path)
    else:
        # Load transformer and transform data
        transformer = joblib.load(save_path)
        X_data_transformed = transformer.transform(X_data)

    # Create transformed DataFrame with column names
    y_df = pd.DataFrame(y, columns=[class_col_name])
    X_data_df = pd.DataFrame(X_data_transformed, columns=X.columns[X.columns != class_col_name])
    return y_df.join(X_data_df)

def run(ctx):
    # Input arguments
    fit_transformer = ctx.fit_transformer
    transformer_path = ctx.transformer_path

    # Specify number of non-data (above) and data (everything else) columns
    num_non_data_cols = 2
    num_data_cols = exa.meta.input_column_count - num_non_data_cols

    df = create_df(ctx, num_non_data_cols, num_data_cols)

    # Transform feature data
    df = transform_dataframe(df, class_col_name='class',
                                fit_transformer=fit_transformer,
                                save_path=transformer_path)

    # Output data
    for i in range(0, df.shape[0]):
        out_list = np_numeric_as_scalar(df.iloc[i, :])
        ctx.emit(*out_list)
/
""")

conn.execute(sql)

# Transform training data
sql = textwrap.dedent(f"""\
CREATE OR REPLACE TABLE IDA.TRAIN_TRANSFORMED AS
    SELECT IDA.DF_TRANSFORM(TRUE, '{transformer_path}', {", ".join(column_names)}) from IDA.TRAIN
""")
conn.execute(sql)

# Transform test data
sql = textwrap.dedent(f"""\
CREATE OR REPLACE TABLE IDA.TEST_TRANSFORMED AS
    SELECT IDA.DF_TRANSFORM(FALSE, '{bucketfs_service}{transformer_path}', {", ".join(column_names)}) from IDA.TEST
""")
conn.execute(sql)

# Close Exasol connection
conn.close()

### Build the Model

Now that we have transformed the data, we will build a model, which will be used to predict if each instance is an APS failure or not.

For this example, we will use a classifier based on the extremely randomized trees (extra-trees) algorithm. This tree-based ensemble method is similar to a random forest, except that the tree splitting is randomized, among other things. This can improve the accuracy as well as the computation time. Details can be found [here](https://orbi.uliege.be/handle/2268/9357).

As with most machine learning algorithms, there are multiple parameters which need to be tuned in order to optimize the performance of the classifier for our problem and data. Rather than try many combinations by hand, we will use grid search and cross validation on the training data to find the optimal parameters in the specified subset of parameters.

Because searching a large grid can be computationally intensive, a good set of parameter values has already been found using grid search offline. Thus, only a small, coarse subspace of the search grid is used in the example code below so that executing the code does not take too long.

<div class="alert alert-info">
A good set of parameter values found offline using grid search is the following:<br>
n_estimators: 61<br>
max_depth: 10<br>
class_weight: {0: 1, 1: 89}
</div>

After the optimal parameters have been found using grid search, an `ExtraTreesClassifier` model will be created using those parameters and saved to an EXABucket for use in the next step, which is training the model.

In [ ]:
# Create Exasol connection
conn = pyexasol.connect(dsn=EXASOL_HOST, user=EXASOL_USER, password=EXASOL_PASSWORD, compression=True)
conn.execute(f"ALTER SESSION SET SCRIPT_LANGUAGES='{EXASOL_SCRIPT_LANGUAGES}'")

# EXABucket path to store the classifier
classifier_path = "/udfs/classifier.pkl"

# Script input column descriptions are now the same as output
# One class label, numeric data
column_types = ["INT"] + ["DOUBLE"] * (len(column_names) - 1)
column_desc = [" ".join(t) for t in zip(column_names, column_types)]

sql = textwrap.dedent(f"""\
CREATE OR REPLACE SKLEARN_PYTHON SET SCRIPT
IDA.BUILD_MODEL(classifier_path VARCHAR(200), {", ".join(column_desc)})
EMITS (dummy int) AS

import numpy as np
import pandas as pd
import pycurl
import uuid

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.externals import joblib
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.utils import resample

# Random state to use for reproducibility
RAND_STATE = 3

# Create DataFrame
def create_df(ctx, start_col, num_cols):
    df = []
    col_names = [col.name for col in exa.meta.input_columns[start_col : num_cols + num_cols]]
    while True:
        row = []
        for i in range(start_col, start_col + num_cols):
            row.append(ctx[i])
        df.append(row)
        if not ctx.next():
            break;
    df = pd.DataFrame(df, columns=col_names)
    return df

# Upload file to EXABucket
def upload_file_bucketfs(model, path):
    temp_file = "/tmp/" + str(uuid.uuid4().hex + ".pkl")
    joblib.dump(model, temp_file, compress=True)

    with open(temp_file, "rb") as f:
        url = "https://{EXASOL_BUCKETFS_USER}:{EXASOL_BUCKETFS_PASSWORD}@{EXASOL_BUCKETFS_HOST}" + path
        curl = pycurl.Curl()
        curl.setopt(pycurl.URL, url)
        curl.setopt(pycurl.SSL_VERIFYPEER, 0)   
        curl.setopt(pycurl.SSL_VERIFYHOST, 0)
        curl.setopt(curl.UPLOAD, 1)
        curl.setopt(curl.READDATA, f)
        curl.perform()
        curl.close()

# Build extra-tree classifier
def build_et_classifier(X, class_col_name, model_path=None):
    # Convert columns to numeric data types
    X = X.apply(pd.to_numeric)

    y = X.loc[:, class_col_name]
    X_data = X.loc[:, X.columns != class_col_name]

    # Create classifier
    clf = ExtraTreesClassifier(random_state=RAND_STATE, n_jobs=-1)

    # Specify parameter search grid
    # The grid size is kept small to reduce the computation time
    # Good values (known from offline grid search) are:
    # 'n_estimators': 61
    # 'max_depth': 10
    # 'class_weight': {{0: 1, 1: 89}}
    param_grid = [{{
        'n_estimators': [30, 61],
        'max_depth': [5, 10],
        'class_weight': [{{0: 1, 1: 89}}]
    }}]

    # Define scoring metric for grid search from problem description
    def ida_score(y, y_pred):
        false_preds = y - y_pred
        num_false_pos = (false_preds < 0).sum()
        num_false_neg = (false_preds > 0).sum()
        return -(num_false_pos * 10 + num_false_neg * 500)

    ida_scorer = make_scorer(ida_score)

    # Search for optimal values in grid using 5-fold cross validation
    grid_search = GridSearchCV(clf, param_grid, cv=5, scoring=ida_scorer, n_jobs=-1)
    grid_search.fit(X_data, y.values.ravel())

    # Create new model with optimal parameter values
    clf = ExtraTreesClassifier(random_state=RAND_STATE, n_jobs=-1,
                                n_estimators=grid_search.best_params_['n_estimators'],
                                max_depth=grid_search.best_params_['max_depth'], 
                                class_weight=grid_search.best_params_['class_weight'])

    # Save classifier to EXABucket
    if model_path:
        upload_file_bucketfs(clf, model_path)

def run(ctx):
    # Input argument
    classifier_path = ctx.classifier_path

    # Specify number of non-data (above) and data (everything else) columns
    num_non_data_cols = 1
    num_data_cols = exa.meta.input_column_count - num_non_data_cols

    df = create_df(ctx, num_non_data_cols, num_data_cols)

    # Shuffle data
    train_set = resample(df, replace=False, random_state=RAND_STATE)

    # Build extra-tree classifier
    build_et_classifier(train_set, class_col_name='class', model_path=classifier_path)
/
""")

conn.execute(sql)

# Build model
sql = textwrap.dedent(f"""\
SELECT IDA.BUILD_MODEL('{classifier_path}', {", ".join(column_names)}) from IDA.TRAIN_TRANSFORMED
""")

conn.execute(sql)

# Close Exasol connection
conn.close()

### Train the Model

After transforming the data and creating the classifier, we will now train it on all the transformed training data. Then, it will be stored in the provided EXABucket for later use during testing.

In [ ]:
# Create Exasol connection
conn = pyexasol.connect(dsn=EXASOL_HOST, user=EXASOL_USER, password=EXASOL_PASSWORD, compression=True)
conn.execute(f"ALTER SESSION SET SCRIPT_LANGUAGES='{EXASOL_SCRIPT_LANGUAGES}'")

# EXABucket path to load/store the classifier
bucketfs_service = "/buckets/bucketfs1"
classifier_path = "/udfs/classifier.pkl"

sql = textwrap.dedent(f"""\
CREATE OR REPLACE SKLEARN_PYTHON SET SCRIPT
IDA.TRAIN_MODEL(classifier_load_path VARCHAR(200), classifier_save_path VARCHAR(200), {", ".join(column_desc)})
EMITS (dummy int) AS

import numpy as np
import pandas as pd
import pycurl
import uuid

from sklearn.externals import joblib
from sklearn.utils import resample

# Random state to use for reproducibility
RAND_STATE = 3

# Create DataFrame
def create_df(ctx, start_col, num_cols):
    df = []
    col_names = [col.name for col in exa.meta.input_columns[start_col : num_cols + num_cols]]
    while True:
        row = []
        for i in range(start_col, start_col + num_cols):
            row.append(ctx[i])
        df.append(row)
        if not ctx.next():
            break;
    df = pd.DataFrame(df, columns=col_names)
    return df

# Upload file to EXABucket
def upload_file_bucketfs(model, path):
    temp_file = "/tmp/" + str(uuid.uuid4().hex + ".pkl")
    joblib.dump(model, temp_file, compress=True)

    with open(temp_file, "rb") as f:
        url = "https://{EXASOL_BUCKETFS_USER}:{EXASOL_BUCKETFS_PASSWORD}@{EXASOL_BUCKETFS_HOST}" + path
        curl = pycurl.Curl()
        curl.setopt(pycurl.URL, url)
        curl.setopt(pycurl.SSL_VERIFYPEER, 0)   
        curl.setopt(pycurl.SSL_VERIFYHOST, 0)
        curl.setopt(curl.UPLOAD, 1)
        curl.setopt(curl.READDATA, f)
        curl.perform()
        curl.close()

# Train classifier
def train(X, class_col_name, model_load_path, model_save_path):
    # Convert columns to numeric data types
    X = X.apply(pd.to_numeric)

    y = X.loc[:, class_col_name]
    X_data = X.loc[:, X.columns != class_col_name]

    # Load model from EXABucket
    clf = joblib.load(model_load_path)
    clf.fit(X_data, y.values.ravel())

    # Save classifier to EXABucket
    if model_save_path:
        upload_file_bucketfs(clf, model_save_path)

def run(ctx):
    # Input argument
    classifier_load_path = ctx.classifier_load_path
    classifier_save_path = ctx.classifier_save_path

    # Specify number of non-data (above) and data (everything else) columns
    num_non_data_cols = 2
    num_data_cols = exa.meta.input_column_count - num_non_data_cols

    df = create_df(ctx, num_non_data_cols, num_data_cols)

    # Shuffle data
    train_set = resample(df, replace=False, random_state=RAND_STATE)

    # Train the classifier
    train(train_set, class_col_name='class',
            model_load_path=classifier_load_path,
            model_save_path=classifier_save_path)
/
""")

conn.execute(sql)

# Train model
sql = textwrap.dedent(f"""\
SELECT IDA.TRAIN_MODEL('{bucketfs_service}{classifier_path}', '{classifier_path}', {", ".join(column_names)}) from IDA.TRAIN_TRANSFORMED
""")

conn.execute(sql)

# Close Exasol connection
conn.close()

### Test the Model

After training the classifier, we will now test it using the transformed test data.

The model that was saved during training will now be loaded from the EXABucket and used to predict the classes of the test data. The returned data, which is stored in the table `TEST_PREDICTIONS`, will be the predicted classes (first column) joined to the transformed test data. In this way, we can ensure that the predicted and real classes remain properly ordered/linked for evaluation.

In [ ]:
# Create Exasol connection
conn = pyexasol.connect(dsn=EXASOL_HOST, user=EXASOL_USER, password=EXASOL_PASSWORD, compression=True)
conn.execute(f"ALTER SESSION SET SCRIPT_LANGUAGES='{EXASOL_SCRIPT_LANGUAGES}'")

# Create script output column descriptions
# Two class labels, numeric data
out_column_types = ["INT"] * 2 + ["DOUBLE"] * (len(column_names) - 1)
out_column_desc = [" ".join(t) for t in zip(["class_pred"] + column_names, out_column_types)]

# EXABucket path to load the classifier
bucketfs_service = "/buckets/bucketfs1"
classifier_path = "/udfs/classifier.pkl"

sql = textwrap.dedent(f"""\
CREATE OR REPLACE SKLEARN_PYTHON SET SCRIPT
IDA.TEST_MODEL(classifier_path VARCHAR(200), {", ".join(column_desc)})
EMITS ({", ".join(out_column_desc)}) AS

import numpy as np
import pandas as pd

from sklearn.externals import joblib

# Create DataFrame
def create_df(ctx, start_col, num_cols):
    df = []
    col_names = [col.name for col in exa.meta.input_columns[start_col : num_cols + num_cols]]
    while True:
        row = []
        for i in range(start_col, start_col + num_cols):
            row.append(ctx[i])
        df.append(row)
        if not ctx.next():
            break;
    df = pd.DataFrame(df, columns=col_names)
    return df

# Convert NumPy values to native Python type
def np_numeric_as_scalar(x):
    return [np.asscalar(x[i]) for i in range(0, len(x))]

# Test classifier
def test(X, class_col_name, model_path=None):
    # Convert columns to numeric data types
    X = X.apply(pd.to_numeric)

    X_data = X.loc[:, X.columns != class_col_name]

    # Load model from EXABucket
    clf = joblib.load(model_path)

    # Predict classes of test data
    return clf.predict(X_data)

def run(ctx):
    # Input argument
    classifier_path = ctx.classifier_path

    # Specify number of non-data (above) and data (everything else) columns
    num_non_data_cols = 1
    num_data_cols = exa.meta.input_column_count - num_non_data_cols

    df = create_df(ctx, num_non_data_cols, num_data_cols)

    # Test the classifier
    y_pred = test(df, class_col_name='class', model_path=classifier_path)

    # Add class predictions as first column of test DataFrame
    df_pred = (pd.DataFrame(y_pred, columns=['class_pred'])).join(df)

    # Convert columns to numeric data types
    df_pred = df_pred.apply(pd.to_numeric)

    # Output data
    for i in range(0, df_pred.shape[0]):
        out_list = np_numeric_as_scalar(df_pred.iloc[i, :])
        ctx.emit(*out_list)
/
""")

conn.execute(sql)

# Test model
sql = textwrap.dedent(f"""\
CREATE OR REPLACE TABLE IDA.TEST_PREDICTIONS AS
    SELECT IDA.TEST_MODEL('{bucketfs_service}{classifier_path}', {", ".join(column_names)}) from IDA.TEST_TRANSFORMED
""")

conn.execute(sql)

# Close Exasol connection
conn.close()

### Evaluate the Model

Now that we have the predicted classes labels of the test data, we can just compare them to the real class labels to evaluate how well our classifier performs.

We use the `ida_cost` method as the performance metric, which was defined in the problem description. Additionally, the confusion is also displayed to see how well each class was classified.

In [ ]:
from sklearn.metrics import confusion_matrix

# Define cost function from the problem description
def ida_cost(y, y_pred):
    false_preds = y - y_pred
    num_false_pos = (false_preds < 0).sum()
    num_false_neg = (false_preds > 0).sum()
    return num_false_pos * 10 + num_false_neg * 500

# Create Exasol connection
conn = pyexasol.connect(dsn=EXASOL_HOST, user=EXASOL_USER, password=EXASOL_PASSWORD, compression=True)

# Get predicted and real class labels
test_preds = conn.export_to_pandas("SELECT CLASS_PRED, CLASS FROM IDA.TEST_PREDICTIONS")

# Close Exasol connection
conn.close()

y_pred = test_preds.loc[:, 'CLASS_PRED']
y = test_preds.loc[:, 'CLASS']

# Examine the results
confusion_mat = confusion_matrix(y, y_pred)
confusion_matrix_df = pd.DataFrame(confusion_mat, index=['neg', 'pos'], columns=['neg_pred', 'pos_pred'])

print("Cost:", ida_cost(y, y_pred),"\n")
print("Confusion Matrix:\n", confusion_matrix_df)

### Summary

Having gone through this simple, real-world example, hopefully you can now see how you can integrate your machine learning algorithms and data science processes directly into Exasol.